# Setup FastSurfer

In [1]:
#@title Here we first setup the environment by downloading the open source deep-mi/fastsurfer project and the required packages
import os
import sys
from os.path import exists, join, basename, splitext

print("Starting setup. This could take a few minutes")
print("----------------------------------------------")

is_google_colab = "colab.research.google.com" in str(os.environ)
if is_google_colab:
    # this is for a Google Colab Notebook
    SETUP_DIR = "/content/"
else:
    SETUP_DIR = os.environ["HOME"] + "/fastsurfer_tutorial/"

# Go to the FastSurfer directory
!mkdir -p "{SETUP_DIR}"
%cd "{SETUP_DIR}"

print(f"Using {SETUP_DIR} to store files.")

print("Downloading FastSurfer")
print("----------------------------------------------")


git_repo_url = 'https://github.com/deep-mi/fastsurfer.git'
project_name = splitext(basename(git_repo_url))[0]
FASTSURFER_HOME = SETUP_DIR + project_name + "/"
if not exists(project_name):
  # clone and install dependencies
  ! git clone -q --branch stable $git_repo_url
  ! pip install -r $FASTSURFER_HOME/requirements.txt
sys.path.append(FASTSURFER_HOME)

# Update dependencies
print("Installing required packages")
print("----------------------------------------------")

! pip install torchio==0.18.83
! pip install yacs==0.1.8
! pip install plotly==5.9.0

print("Finished setup")
print("----------------------------------------------")


Starting setup. This could take a few minutes
----------------------------------------------
/content
Using /content/ to store files.
----------------------------------------------
Installing required packages
----------------------------------------------
Finished setup
----------------------------------------------


# Load Data

In [2]:
part = 1

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!rm -rf sample_data

In [5]:
# !ls "/content/drive/MyDrive/Data/ADNI1-1YR-1.5T/standard" | grep ".zip" | head -n 1 | while read f; do cp "/content/drive/MyDrive/Data/ADNI1-1YR-1.5T/standard/$f" .; done
!cp "/content/drive/MyDrive/Data/ADNI1-1YR-1.5T/standard/ADNI1:Complete 1Yr 1.5T""$part"".zip" .

In [6]:
!rm -rf ADNI && mkdir ADNI

In [7]:
!ls . | grep .zip | while read f; do echo $f && unzip "$f" > /dev/null; done

ADNI1:Complete 1Yr 1.5T1.zip


In [3]:
# !rm -rf fastsurfer_seg

In [4]:
# !cp /content/drive/MyDrive/Data/ADNI1-1YR-1.5T/statistics/seg_stats_p1.tar.gz .

In [5]:
# !tar -xzf seg_stats_p1.tar.gz && rm seg_stats_p1.tar.gz

# Run FastSurfer

In [3]:
import glob

nifties = sorted(glob.glob('ADNI/**/*.nii', recursive=True))

In [4]:
# find out how many samples are already processed
batch_size = 30

# batch_number = 0
batch_number = len(glob.glob(f"/content/drive/MyDrive/Data/ADNI1-1YR-1.5T/statistics/seg_stats_p{part}*.tar.gz"))

# start_index = batch_number * batch_size + 0
start_index = 50

In [ ]:
index = start_index

while index < len(nifties):

  img = nifties[index]
  sid = img.split('/')[1] + '__' + img.split('/')[3]
  image="/content/" + img

  # Note, you should also add --3T, if you are processing data from a 3T scanner.
  !FASTSURFER_HOME=$FASTSURFER_HOME && bash $FASTSURFER_HOME/run_fastsurfer.sh --t1 "$image" \
                                      --sd "{SETUP_DIR}fastsurfer_seg" \
                                      --sid $sid \
                                      --seg_only --py python3 \
                                      --allow_root

  !rm -rf fastsurfer_seg/"$sid"/scripts
  !rm -rf fastsurfer_seg/"$sid"/mri

  if index % batch_size == batch_size - 1:
    # archive and save
    !tar -czf seg_stats_p"$part""$batch_number".tar.gz fastsurfer_seg
    !cp seg_stats_p"$part""$batch_number".tar.gz /content/drive/MyDrive/Data/ADNI1-1YR-1.5T/statistics/

    # clear all of the fastsurfer_seg folder
    !rm -rf fastsurfer_seg
    !rm seg_stats_p"$part""$batch_number".tar.gz

    # change batch number
    batch_number += 1

  index += 1


if index % batch_size != 0:
  # archive and save
  !tar -czf seg_stats_p"$part""$batch_number".tar.gz fastsurfer_seg
  !cp seg_stats_p"$part""$batch_number".tar.gz /content/drive/MyDrive/Data/ADNI1-1YR-1.5T/statistics/

  # clear all of the fastsurfer_seg folder
  !rm -rf fastsurfer_seg
  !rm seg_stats_p"$part""$batch_number".tar.gz


Setting ENV variable FASTSURFER_HOME to script directory /content/fastsurfer. 
Change via environment to location of your choice if this is undesired (export FASTSURFER_HOME=/dir/to/FastSurfer)
Version: 2.2.0+a000faa
Sun Mar 31 05:15:42 PM UTC 2024

python3 /content/fastsurfer/FastSurferCNN/run_prediction.py --t1 /content/ADNI/021_S_0337/MPR__GradWarp__N3__Scaled/2006-03-28_10_05_12.0/I33437/ADNI_021_S_0337_MR_MPR__GradWarp__N3__Scaled_Br_20061217123842517_S12485_I33437.nii --asegdkt_segfile /content/fastsurfer_seg/021_S_0337__2006-03-28_10_05_12.0/mri/aparc.DKTatlas+aseg.deep.mgz --conformed_name /content/fastsurfer_seg/021_S_0337__2006-03-28_10_05_12.0/mri/orig.mgz --brainmask_name /content/fastsurfer_seg/021_S_0337__2006-03-28_10_05_12.0/mri/mask.mgz --aseg_name /content/fastsurfer_seg/021_S_0337__2006-03-28_10_05_12.0/mri/aseg.auto_noCCseg.mgz --sid 021_S_0337__2006-03-28_10_05_12.0 --seg_log /content/fastsurfer_seg/021_S_0337__2006-03-28_10_05_12.0/scripts/deep-seg.log --vox_size 

In [ ]:
!tar -czf seg_stats_p"$part".tar.gz fastsurfer_seg

In [ ]:
!cp seg_stats_p1.tar.gz /content/drive/MyDrive/Data/ADNI1-1YR-1.5T/statistics/